In [1]:
from deepgram import DeepgramClient, PrerecordedOptions, FileSource
import requests

import os
from dotenv import load_dotenv
import yt_dlp as youtube_dl

In [2]:
load_dotenv()

#mp4 directory
SAVE_DIR = '/'.join(os.getcwd().split('/')[:3]) + '/speechdir'
# Deepgram API key
DG_KEY = os.getenv('DEEPGRAM_API_KEY')

AUDIO_PATH = SAVE_DIR + '/audio_cont'


ydl_opts = {
  'format': 'best',
  'outtmpl': AUDIO_PATH + '/%(title)s.%(ext)s',
  'noplaylist': True,
  'extract-audio': True,
}

video = "https://www.youtube.com/watch?v=PeMlggyqz0Y"

In [3]:
def process_download(video_url: str = None, ydl_opts: dict = ydl_opts):
    #if in SAVE_PATH have another file, delete it to be empty
	if os.path.exists(AUDIO_PATH):
		for file in os.listdir(AUDIO_PATH):
			os.remove(AUDIO_PATH + '/' + file)
	with youtube_dl.YoutubeDL(ydl_opts) as ydl:
		info_dict = ydl.extract_info(video_url, download=True)
		video_url = info_dict.get("url", None)
		video_title = info_dict.get('title', None)
		video_length = info_dict.get('duration')
	return video_title, video_length

In [4]:
video_title, video_length = process_download(video, ydl_opts)

[youtube] Extracting URL: https://www.youtube.com/watch?v=PeMlggyqz0Y
[youtube] PeMlggyqz0Y: Downloading webpage
[youtube] PeMlggyqz0Y: Downloading tv client config
[youtube] PeMlggyqz0Y: Downloading player c8dbda2a
[youtube] PeMlggyqz0Y: Downloading tv player API JSON
[youtube] PeMlggyqz0Y: Downloading ios player API JSON
[youtube] PeMlggyqz0Y: Downloading m3u8 information
[info] PeMlggyqz0Y: Downloading 1 format(s): 18
[download] Destination: d:\Project\ThesisProd\notebooks\speechdir\audio_cont\Machine Learning Explained in 100 Seconds.mp4
[download] 100% of    3.99MiB in 00:00:00 at 8.31MiB/s     


In [5]:
JSON_PATH = SAVE_DIR + '/json_cont'
# URL of the audio file
AUDIO_FILE = AUDIO_PATH + '/' + video_title + '.mp4'
# Path to save the transcript JSON file
TRANSCRIPT_FILE = JSON_PATH + '/' + video_title + '.json'

In [6]:
def transcript():
    try:
        # STEP 1: Create a Deepgram client using the API key
        deepgram = DeepgramClient(DG_KEY)

        # Download the audio file from the URL
        if os.path.exists(JSON_PATH):
            for file in os.listdir(JSON_PATH):
                os.remove(JSON_PATH + '/' + file)

        with open(AUDIO_FILE, "rb") as file:
            buffer_data = file.read()

        payload: FileSource = {
            "buffer": buffer_data,
        }

        # STEP 2: Configure Deepgram options for audio analysis
        options = PrerecordedOptions(
            model="nova-2",
            smart_format=True,
        )

        # STEP 3: Call the transcribe_file method with the text payload and options
        response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)

        # STEP 4: Write the response JSON to a file
        with open(TRANSCRIPT_FILE, "w") as transcript_file:
            transcript_file.write(response.to_json(indent=4))

        print("Transcript JSON file generated successfully.")

    except Exception as e:
        print(f"Exception: {e}")

In [7]:
transcript()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_23564\839383143.py:25: DeprecatedWarning: prerecorded is deprecated as of 3.4.0 and will be removed in 4.0.0. deepgram.listen.prerecorded is deprecated. Use deepgram.listen.rest instead.
  response = deepgram.listen.prerecorded.v("1").transcribe_file(payload, options)


Transcript JSON file generated successfully.


In [11]:
import json
def print_transcript(transcription_file: str = None):
    with open(transcription_file, "r") as file:
        final_res = ""
        data = json.load(file)
        result = data['results']['channels'][0]['alternatives'][0]['transcript']
        result = result.split('.')
        for sentence in result:
            final_res += sentence + '. '
            # print(sentence + '.')
        return final_res

In [12]:
result  = print_transcript(TRANSCRIPT_FILE)

In [13]:
result

"Machine learning.  Teach a computer how to perform a task without explicitly programming it to perform said task.  Instead, feed data into an algorithm to gradually improve outcomes with experience, similar to how organic life learns.  The term was coined in 1959 by Arthur Samuel at IBM, who was developing artificial intelligence that could play checkers.  Half a century later, and predictive models are embedded in many of the products we use every day, which perform two fundamental jobs.  One is to classify data, like is there another car on the road, or does this patient have cancer? The other is to make predictions about future outcomes like will the stock go up or which YouTube video do you wanna watch next? The first step in the process is to acquire and clean up data.  Lots and lots of data.  The better the data represents the problem, the better the results.  Garbage in, garbage out.  The data needs to have some kind of signal to be valuable to the algorithm for making predicti